In [17]:
import onnxruntime as ort

onnx_model_path = "hair_classifier_v1.onnx"
session = ort.InferenceSession(onnx_model_path, providers=["CPUExecutionProvider"])

In [18]:
inputs = session.get_inputs()
outputs = session.get_outputs()

input_name = inputs[0].name
output_name = outputs[0].name

In [19]:
output_name

'output'

In [21]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [22]:
!wget https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg -O hair_image.jpeg

--2025-12-02 08:27:53--  https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg
Resolving habrastorage.org (habrastorage.org)... 95.47.173.35, 95.47.173.34, 2a14:b680:0:56::34, ...
Connecting to habrastorage.org (habrastorage.org)|95.47.173.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 398272 (389K) [image/jpeg]
Saving to: ‘hair_image.jpeg’

hair_image.jpeg     100%[===================>] 388.94K   699KB/s    in 0.6s    

2025-12-02 08:27:55 (699 KB/s) - ‘hair_image.jpeg’ saved [398272/398272]



## 200,200

In [23]:
!pip install keras-image-helper

In [24]:
import numpy as np
from keras_image_helper import create_preprocessor

def preprocess_pytorch(X):
    # X: shape (1, 299, 299, 3), dtype=float32, values in [0, 255]
    X = X / 255.0

    mean = np.array([0.485, 0.456, 0.406]).reshape(1, 3, 1, 1)
    std = np.array([0.229, 0.224, 0.225]).reshape(1, 3, 1, 1)

    # Convert NHWC → NCHW
    # from (batch, height, width, channels) → (batch, channels, height, width)
    X = X.transpose(0, 3, 1, 2)

    # Normalize
    X = (X - mean) / std

    return X.astype(np.float32)


preprocessor = create_preprocessor(preprocess_pytorch, target_size=(200, 200))

In [25]:
url = 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'
X = preprocessor.from_url(url)
X.shape

(1, 3, 200, 200)

In [26]:
first_pixel_R = X[0, 0, 0, 0]
print("First pixel R value:", first_pixel_R)

First pixel R value: -1.0732939


In [27]:
result = session.run([output_name], {input_name: X})

In [28]:
result

[array([[0.09156627]], dtype=float32)]

## 608MB

## -0.1